In [1]:
import csv
import json
import os
from typing import IO, TextIO

import pandas as pd

from dadis_client.client import DadisClient

In [2]:
API_KEY = os.environ["DADIS_API_KEY"]

In [3]:
client = DadisClient(api_key=API_KEY, prod=True)

In [4]:
def full_matching_workflow(
    input_filename: str,
    output_filename: str,
    dadis_api_key: str) -> pd.DataFrame:
    """
    Perform the full matching workflow:

    - Read VBO data from input_filename
    - Match to DADIS to get dadis_transboundary_id
    - Save to a new TSV file at output_filename
    """
    client = DadisClient(api_key=dadis_api_key)
    vbo_data = read_vbo_data(input_filename)
    matched_breeds = match_vbo_breeds(vbo_data=vbo_data, client=client)

    print(f"Writing output file to {output_filename}:")
    output_file = create_output_tsv(input_filename=input_filename, output_filename=output_filename, extra_cols=["dadis_transboundary_id"])
    matched_breeds.to_csv(output_file, sep="\t", index=False, header=False)
    output_file.close()
    print("Output written.")
    return matched_breeds
    

def read_vbo_data(filename: str) -> pd.DataFrame:
    df = pd.read_table(filename, skiprows=[1]).convert_dtypes()
    print("Dropping duplicate entry for: VBO:0000991 - Icelandic Horse/Iceland Pony")
    dup_entry = (df["VBO id"] == "VBO:0000991") & (df["term label"].isna())
    return df.loc[~ dup_entry, :]


def get_dadis_species(client: DadisClient) -> pd.DataFrame:
    resp = client.get_all_species()
    all_species = []
    for s in resp.response:
        species = {"dadis_species_id": s.id, "species_name": s.name["en"]}
        all_species.append(species)
    return pd.DataFrame.from_records(all_species)


def get_canonical_dadis_transboundary(client: DadisClient) -> pd.DataFrame:
    """
    DADIS has a canonical name for each transboundary breed, fetch these
    and return them as a dataframe
    """
    resp = client.get_all_transboundary_names()
    df = (
        pd.DataFrame.from_records([b.model_dump() for b in resp.response])
        .rename(columns={"speciesId": "dadis_species_id"})
    )
    species_df = get_dadis_species(client)
    df = df.merge(species_df, how="left", on="dadis_species_id")
    df = df.rename(columns={"id": "dadis_transboundary_id", "name": "dadis_breed_name"})
    return df

def get_all_dadis_transboundary(client: DadisClient) -> pd.DataFrame:
    """
    Get all names for DADIS transboundary breeds, some VBO entries
    may use non-canonical names
    """
    resp = client.get_all_transboundary_breeds()
    df = (
        pd.DataFrame.from_records([b.model_dump() for b in resp.response])
        .rename(columns={
            "speciesId": "dadis_species_id", 
            "name": "dadis_breed_name", 
            "transboundaryId": "dadis_transboundary_id",
            "id": "dadis_breed_id",
            "iso3": "dadis_iso3_code",
        })
        .drop_duplicates(subset=["dadis_species_id", "dadis_breed_name", "dadis_transboundary_id"])
    )
    species_df = get_dadis_species(client)
    result = (
        df
        .merge(species_df, how="left", on="dadis_species_id")
        .sort_values(["dadis_transboundary_id", "dadis_breed_name"])
    )
    return result


def get_simple_matches(vbo_data: pd.DataFrame, client: DadisClient) -> pd.DataFrame:
    """
    Match VBO entries to DADIS transboundary breeds based on their
    canonical names. Return a dataframe containing the matches
    """
    dadis_canonical = get_canonical_dadis_transboundary(client=client)
    simple_matches = (
        vbo_data[["VBO id", "term label", "DADIS_name", "DADIS_species_name"]].merge(
            dadis_canonical,
            how="left",
            left_on=["DADIS_name", "DADIS_species_name"],
            right_on=["dadis_breed_name", "species_name"],
            sort=False,
        )
        .drop(columns=["dadis_breed_name", "species_name"])
        .convert_dtypes()
    )
    return simple_matches


def get_extra_matches(vbo_data: pd.DataFrame, client: DadisClient) -> pd.DataFrame:
    dadis_all = get_all_dadis_transboundary(client=client)
    extra_matches = (
        vbo_data[["VBO id", "term label", "DADIS_name", "DADIS_species_name"]].merge(
            dadis_all,
            how="left",
            left_on=["DADIS_name", "DADIS_species_name"],
            right_on=["dadis_breed_name", "species_name"],
            indicator=True,
            # Need to ensure we use sort=False so order stays consistent with original
            sort=False,
        )
        .drop(columns=["dadis_breed_name", "species_name"])
        .convert_dtypes()
        .drop_duplicates(subset=["VBO id", "dadis_transboundary_id"])
    )
    counts = extra_matches["VBO id"].value_counts()
    duplicates = counts.loc[counts >=2 ].index.tolist()
    multiple_matches = extra_matches.loc[extra_matches["VBO id"].isin(duplicates), :]
    print("The following entries matched against multiple DADIS entries - will not be updated")
    print(multiple_matches["VBO id"])
    extra_matches = extra_matches.loc[~ extra_matches["VBO id"].isin(duplicates), :]
    return extra_matches


def match_vbo_breeds(vbo_data: pd.DataFrame, client: DadisClient) -> pd.DataFrame:
    """
    Match VBO entries to DADIS, based on both canonical name (preferred) or
    other transboundary name.

    Return a modified copy of vbo_data, with the 'dadis_transboundary_id' added
    """
    print("Matching to canonical DADIS names")
    simple_matches = get_simple_matches(vbo_data, client)
    print("Matching to other DADIS names")
    extra_matches = get_extra_matches(vbo_data, client)
    all_matches = simple_matches.merge(
        extra_matches[["VBO id", "dadis_transboundary_id", "dadis_species_id"]],
        how="left",
        on="VBO id",
        suffixes=(None, "_extra"),
        sort=False
    ).convert_dtypes()
    # Fill in simple matches with extra matches, where no simple match was found
    simple_ids = all_matches["dadis_transboundary_id"]
    extra_ids = all_matches["dadis_transboundary_id_extra"]
    all_matches["dadis_transboundary_id"] = simple_ids.where(~ simple_ids.isna(), extra_ids)

    n_total = all_matches.shape[0]
    n_matched = all_matches["dadis_transboundary_id"].notna().sum()
    print(f"{n_matched} / {n_total} VBO entries matched with DADIS")
    
    return vbo_data.merge(
        all_matches[["VBO id", "dadis_transboundary_id"]],
        on="VBO id",
        how="left",
        sort=False
    )


def create_output_tsv(input_filename: str, output_filename: str, extra_cols: list[str] = None) -> TextIO:
    """
    Copy the 2 header lines from the input file to the output file. Return
    a file object for the output file, so pandas can write the rest of the file
    """
    file_out = open(output_filename, "w")
    csv_out = csv.writer(file_out, dialect="excel-tab")
    with open(input_filename) as file_in:
        csv_in = csv.reader(file_in, dialect="excel-tab")
        for index, line in enumerate(range(2)):
            header = next(csv_in)
            if extra_cols is not None:
                if index == 0:
                    header += extra_cols
                if index == 1:
                    header += ['' for i in range(len(extra_cols))]
            csv_out.writerow(header)
    return file_out

# Test full updating process

In [5]:
result = full_matching_workflow(input_filename="data/dadistransbound.tsv", output_filename="example_transboundary_output.tsv", dadis_api_key=API_KEY)

Dropping duplicate entry for: VBO:0000991 - Icelandic Horse/Iceland Pony
Matching to canonical DADIS names
Matching to other DADIS names
The following entries matched against multiple DADIS entries - will not be updated
19      VBO:0000056
20      VBO:0000056
67      VBO:0000104
68      VBO:0000104
144     VBO:0000184
145     VBO:0000184
168     VBO:0000207
169     VBO:0000207
177     VBO:0000215
178     VBO:0000215
227     VBO:0000265
228     VBO:0000265
235     VBO:0000272
236     VBO:0000272
279     VBO:0000315
280     VBO:0000315
286     VBO:0000321
287     VBO:0000321
355     VBO:0000389
356     VBO:0000389
389     VBO:0000422
390     VBO:0000422
414     VBO:0000446
415     VBO:0000446
614     VBO:0000645
615     VBO:0000645
677     VBO:0000717
678     VBO:0000717
711     VBO:0000751
712     VBO:0000751
911     VBO:0000951
912     VBO:0000951
1107    VBO:0001146
1108    VBO:0001146
1123    VBO:0001161
1124    VBO:0001161
1212    VBO:0001249
1213    VBO:0001249
1217    VBO:0001253


In [6]:
result.head()

,VBO id,term label,breed name to be used for label,species to be used for label,internal term merge indicator vboid into vboid,internal date and issue separated by pipe YYMMDD-issue,GH issue NOT merge obsolete,parent ID,source for parents,source for parents.1,...,replacement label,comment.1,GH issue,obsolescence reason,curator note,DADIS_name,DADIS_species_name,DADIS_country,DADIS_ISO3_code,dadis_transboundary_id
0,VBO:0000012,Dromedary Bactrian Camel,<NA>,<NA>,<NA>,<NA>,<NA>,NCBITaxon:9836,<NA>,DOI:10.1016/j.jasrep.2021.103038|PMID:31969478...,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,VBO:0000038,Alpaca (Alpaca),Alpaca,Alpaca,<NA>,<NA>,<NA>,NCBITaxon:30538,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,Alpaca,Alpaca,<NA>,<NA>,<NA>
2,VBO:0000039,Huacaya (Alpaca),Huacaya,Alpaca,<NA>,<NA>,<NA>,NCBITaxon:30538,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,Huacaya,Alpaca,<NA>,<NA>,1-1
3,VBO:0000040,Suri (Alpaca),Suri,Alpaca,<NA>,<NA>,<NA>,NCBITaxon:30538,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,Suri,Alpaca,<NA>,<NA>,1-2
4,VBO:0000041,American Bison (American Bison),American Bison,American Bison,<NA>,<NA>,<NA>,NCBITaxon:9901,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,American Bison,American Bison,<NA>,<NA>,2-1


# Step-by-step process: for debugging

Note: may not be fully up to date with the process above.

In [7]:
dadis_species = get_dadis_species(client)
dadis_species.head()

,dadis_species_id,species_name
0,1,Alpaca
1,2,American Bison
2,3,Ass
3,4,Bactrian camel
4,5,Buffalo


In [8]:
dadis_canonical = get_canonical_dadis_transboundary(client=client)
dadis_canonical.head()

,dadis_transboundary_id,dadis_species_id,dadis_breed_name,species_name
0,1-1,1,Huacaya,Alpaca
1,1-2,1,Suri,Alpaca
2,2-1,2,American Bison,American Bison
3,3-1,3,Asino Sardo,Ass
4,3-2,3,Balkan Donkey,Ass


In [9]:
dadis_all = get_all_dadis_transboundary(client=client)
dadis_all.head()

,dadis_breed_id,dadis_breed_name,dadis_iso3_code,dadis_species_id,dadis_transboundary_id,updatedAt,species_name
147,da0f5e4b-2dc4-4119-bb9a-6d87d88b2a3c,Huacaya,AUS,1,1-1,1652172491000,Alpaca
148,32354ff8-1f74-4e90-abb5-657753427508,Suri,AUS,1,1-2,1652172491000,Alpaca
617,56378f94-0804-411b-a7bd-a9b0a43bf187,Sury,BOL,1,1-2,1665583332000,Alpaca
1091,ba06f2a8-caca-4dce-8c22-8599b349b85e,Test,DEU,1,1-2,1678781404000,Alpaca
242,0c273dfe-f832-431b-938a-b9235a6dbffc,Chital,AUS,11,11-1,1652172493000,Deer


In [10]:
dadis_all.dtypes

dadis_breed_id            object
dadis_breed_name          object
dadis_iso3_code           object
dadis_species_id           int64
dadis_transboundary_id    object
updatedAt                  int64
species_name              object
dtype: object

## VBO transboundary data

In [11]:
VBO_TRANSBOUNDARY_FILENAME = "data/dadistransbound.tsv"

In [12]:
vbo_transboundary = pd.read_table(VBO_TRANSBOUNDARY_FILENAME, skiprows=[1]).convert_dtypes()
# Drop problematic entry: VBO:0000991 - Icelandic Horse/Iceland Pony
dup_entry = (vbo_transboundary["VBO id"] == "VBO:0000991") & (vbo_transboundary["term label"].isna())
vbo_transboundary = vbo_transboundary.loc[~ dup_entry, :]
vbo_transboundary.head()

,VBO id,term label,breed name to be used for label,species to be used for label,internal term merge indicator vboid into vboid,internal date and issue separated by pipe YYMMDD-issue,GH issue NOT merge obsolete,parent ID,source for parents,source for parents.1,...,replacement term,replacement label,comment.1,GH issue,obsolescence reason,curator note,DADIS_name,DADIS_species_name,DADIS_country,DADIS_ISO3_code
0,VBO:0000012,Dromedary Bactrian Camel,<NA>,<NA>,<NA>,<NA>,<NA>,NCBITaxon:9836,<NA>,DOI:10.1016/j.jasrep.2021.103038|PMID:31969478...,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,VBO:0000038,Alpaca (Alpaca),Alpaca,Alpaca,<NA>,<NA>,<NA>,NCBITaxon:30538,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Alpaca,Alpaca,<NA>,<NA>
2,VBO:0000039,Huacaya (Alpaca),Huacaya,Alpaca,<NA>,<NA>,<NA>,NCBITaxon:30538,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Huacaya,Alpaca,<NA>,<NA>
3,VBO:0000040,Suri (Alpaca),Suri,Alpaca,<NA>,<NA>,<NA>,NCBITaxon:30538,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Suri,Alpaca,<NA>,<NA>
4,VBO:0000041,American Bison (American Bison),American Bison,American Bison,<NA>,<NA>,<NA>,NCBITaxon:9901,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,American Bison,American Bison,<NA>,<NA>


In [13]:
vbo_transboundary.loc[vbo_transboundary["VBO id"] == "VBO:0000991", :]

,VBO id,term label,breed name to be used for label,species to be used for label,internal term merge indicator vboid into vboid,internal date and issue separated by pipe YYMMDD-issue,GH issue NOT merge obsolete,parent ID,source for parents,source for parents.1,...,replacement term,replacement label,comment.1,GH issue,obsolescence reason,curator note,DADIS_name,DADIS_species_name,DADIS_country,DADIS_ISO3_code
936,VBO:0000991,Icelandic Horse (Horse),Icelandic Horse,Horse,VBO:0000990 into VBO:0000991,230207-75,<NA>,NCBITaxon:9796,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Icelandic Horse,Horse,<NA>,<NA>


In [14]:
vbo_transboundary.shape

(1673, 89)

In [15]:
vbo_matched = match_vbo_breeds(vbo_data=vbo_transboundary, client=client)
print(vbo_matched.shape)
vbo_matched.head()

Matching to canonical DADIS names
Matching to other DADIS names
The following entries matched against multiple DADIS entries - will not be updated
19      VBO:0000056
20      VBO:0000056
67      VBO:0000104
68      VBO:0000104
144     VBO:0000184
145     VBO:0000184
168     VBO:0000207
169     VBO:0000207
177     VBO:0000215
178     VBO:0000215
227     VBO:0000265
228     VBO:0000265
235     VBO:0000272
236     VBO:0000272
279     VBO:0000315
280     VBO:0000315
286     VBO:0000321
287     VBO:0000321
355     VBO:0000389
356     VBO:0000389
389     VBO:0000422
390     VBO:0000422
414     VBO:0000446
415     VBO:0000446
614     VBO:0000645
615     VBO:0000645
677     VBO:0000717
678     VBO:0000717
711     VBO:0000751
712     VBO:0000751
911     VBO:0000951
912     VBO:0000951
1107    VBO:0001146
1108    VBO:0001146
1123    VBO:0001161
1124    VBO:0001161
1212    VBO:0001249
1213    VBO:0001249
1217    VBO:0001253
1218    VBO:0001253
1423    VBO:0001461
1424    VBO:0001461
1504    VBO:0

,VBO id,term label,breed name to be used for label,species to be used for label,internal term merge indicator vboid into vboid,internal date and issue separated by pipe YYMMDD-issue,GH issue NOT merge obsolete,parent ID,source for parents,source for parents.1,...,replacement label,comment.1,GH issue,obsolescence reason,curator note,DADIS_name,DADIS_species_name,DADIS_country,DADIS_ISO3_code,dadis_transboundary_id
0,VBO:0000012,Dromedary Bactrian Camel,<NA>,<NA>,<NA>,<NA>,<NA>,NCBITaxon:9836,<NA>,DOI:10.1016/j.jasrep.2021.103038|PMID:31969478...,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,VBO:0000038,Alpaca (Alpaca),Alpaca,Alpaca,<NA>,<NA>,<NA>,NCBITaxon:30538,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,Alpaca,Alpaca,<NA>,<NA>,<NA>
2,VBO:0000039,Huacaya (Alpaca),Huacaya,Alpaca,<NA>,<NA>,<NA>,NCBITaxon:30538,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,Huacaya,Alpaca,<NA>,<NA>,1-1
3,VBO:0000040,Suri (Alpaca),Suri,Alpaca,<NA>,<NA>,<NA>,NCBITaxon:30538,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,Suri,Alpaca,<NA>,<NA>,1-2
4,VBO:0000041,American Bison (American Bison),American Bison,American Bison,<NA>,<NA>,<NA>,NCBITaxon:9901,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,American Bison,American Bison,<NA>,<NA>,2-1


In [16]:
vbo_matched["VBO id"].value_counts()

VBO id
VBO:0000012    1
VBO:0000142    1
VBO:0000040    1
VBO:0000041    1
VBO:0000042    1
              ..
VBO:0016849    1
VBO:0016850    1
VBO:0016851    1
VBO:0016852    1
VBO:0016853    1
Name: count, Length: 1673, dtype: Int64

### Test TSV output with double header

In [17]:
# Create file with just the 2 header lines
output_tsv = create_output_tsv(input_filename=VBO_TRANSBOUNDARY_FILENAME, output_filename="output_test.tsv", extra_cols=["dadis_transboundary_id"])
# Write the DF from pandas, not including the header
vbo_transboundary.to_csv(output_tsv, sep="\t", index=False, header=False)

### Simple matches: direct match to canonical DADIS name

In [18]:
simple_matches = vbo_transboundary[["VBO id", "term label", "DADIS_name", "DADIS_species_name"]].merge(
    dadis_canonical,
    how="left",
    left_on=["DADIS_name", "DADIS_species_name"],
    right_on=["dadis_breed_name", "species_name"],
    indicator=True,
    # Need to ensure we use sort=False so order stays consistent with original
    sort=False,
).drop(columns=["dadis_breed_name", "species_name"]).convert_dtypes()
simple_matches.head()

,VBO id,term label,DADIS_name,DADIS_species_name,dadis_transboundary_id,dadis_species_id,_merge
0,VBO:0000012,Dromedary Bactrian Camel,<NA>,<NA>,<NA>,<NA>,left_only
1,VBO:0000038,Alpaca (Alpaca),Alpaca,Alpaca,<NA>,<NA>,left_only
2,VBO:0000039,Huacaya (Alpaca),Huacaya,Alpaca,1-1,1,both
3,VBO:0000040,Suri (Alpaca),Suri,Alpaca,1-2,1,both
4,VBO:0000041,American Bison (American Bison),American Bison,American Bison,2-1,2,both


In [19]:
simple_matches["_merge"].value_counts()

_merge
both          1009
left_only      664
right_only       0
Name: count, dtype: int64

In [20]:
simple_matches["VBO id"].value_counts()

VBO id
VBO:0000012    1
VBO:0000142    1
VBO:0000040    1
VBO:0000041    1
VBO:0000042    1
              ..
VBO:0016849    1
VBO:0016850    1
VBO:0016851    1
VBO:0016852    1
VBO:0016853    1
Name: count, Length: 1673, dtype: Int64

In [21]:
simple_matches.shape

(1673, 7)

## Additional matching: match to non-canonical DADIS names

In [22]:
extra_matches = vbo_transboundary[["VBO id", "term label", "DADIS_name", "DADIS_species_name"]].merge(
    dadis_all,
    how="left",
    left_on=["DADIS_name", "DADIS_species_name"],
    right_on=["dadis_breed_name", "species_name"],
    indicator=True,
    # Need to ensure we use sort=False so order stays consistent with original
    sort=False,
).drop(columns=["dadis_breed_name", "species_name"])
extra_matches.head()

,VBO id,term label,DADIS_name,DADIS_species_name,dadis_breed_id,dadis_iso3_code,dadis_species_id,dadis_transboundary_id,updatedAt,_merge
0,VBO:0000012,Dromedary Bactrian Camel,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,left_only
1,VBO:0000038,Alpaca (Alpaca),Alpaca,Alpaca,NaN,NaN,NaN,NaN,NaN,left_only
2,VBO:0000039,Huacaya (Alpaca),Huacaya,Alpaca,da0f5e4b-2dc4-4119-bb9a-6d87d88b2a3c,AUS,1.0,1-1,1.652172e+12,both
3,VBO:0000040,Suri (Alpaca),Suri,Alpaca,32354ff8-1f74-4e90-abb5-657753427508,AUS,1.0,1-2,1.652172e+12,both
4,VBO:0000041,American Bison (American Bison),American Bison,American Bison,NaN,NaN,NaN,NaN,NaN,left_only


In [23]:
dup_matches = extra_matches["VBO id"].value_counts().pipe(lambda x: x.loc[x >= 2]).index.tolist()
dup_matches

['VBO:0001253',
 'VBO:0000422',
 'VBO:0001461',
 'VBO:0000321',
 'VBO:0000272',
 'VBO:0000207',
 'VBO:0000104',
 'VBO:0000265',
 'VBO:0001249',
 'VBO:0000951',
 'VBO:0000184',
 'VBO:0000315',
 'VBO:0000215',
 'VBO:0000056',
 'VBO:0000751',
 'VBO:0000645',
 'VBO:0001146',
 'VBO:0001541',
 'VBO:0000446',
 'VBO:0001161',
 'VBO:0000389',
 'VBO:0000717']

In [24]:
extra_matches.loc[extra_matches["VBO id"].isin(dup_matches), :].drop_duplicates(subset=["VBO id", "dadis_transboundary_id"])

,VBO id,term label,DADIS_name,DADIS_species_name,dadis_breed_id,dadis_iso3_code,dadis_species_id,dadis_transboundary_id,updatedAt,_merge
19,VBO:0000056,Native of North Africa (Ass),Native of North Africa,Ass,80cb2410-d98c-4d27-8caf-4d7b3277c5fe,MLI,3.0,3-12,1.639731e+12,both
20,VBO:0000056,Native of North Africa (Ass),Native of North Africa,Ass,a352977b-46e3-48b1-a451-a7763eef50c2,SEN,3.0,3-17,1.639732e+12,both
67,VBO:0000104,Angus (Cattle),Angus,Cattle,3b6861eb-4ef6-4dea-9071-29b0c204f308,AUS,7.0,7-2,1.666926e+12,both
68,VBO:0000104,Angus (Cattle),Angus,Cattle,e13ccc87-28ba-4197-904e-0506824c624c,HRV,7.0,7-7,1.652173e+12,both
144,VBO:0000184,Dahomey (Cattle),Dahomey,Cattle,fddd374d-e930-4a71-bae0-6ecfc9c1d16a,COD,7.0,7-123,1.652173e+12,both
145,VBO:0000184,Dahomey (Cattle),Dahomey,Cattle,fcff27c9-256a-44ea-9b75-cd2b2dc5719c,NLD,7.0,7-69,1.675160e+12,both
168,VBO:0000207,Galloway (Cattle),Galloway,Cattle,35f26b3a-689c-435f-b50a-0cba8018c0f0,NOR,7.0,7-40,1.687858e+12,both
169,VBO:0000207,Galloway (Cattle),Galloway,Cattle,b192cee5-00da-4585-ba6c-1921a6cf2071,AUS,7.0,7-85,1.666928e+12,both
177,VBO:0000215,Gir (Cattle),Gir,Cattle,c5846f95-7c48-4a9b-ac3c-18b647123b69,BRA,7.0,7-70,1.668051e+12,both
178,VBO:0000215,Gir (Cattle),Gir,Cattle,02f31ff8-1076-4451-876a-da86974a8999,AUS,7.0,7-89,1.667361e+12,both


In [25]:
vbo_transboundary.shape

(1673, 89)

In [26]:
extra_matches.shape

(1695, 10)

In [27]:
all_matches = simple_matches.merge(
    extra_matches[["VBO id", "dadis_transboundary_id", "dadis_species_id"]],
    how="left",
    on="VBO id",
    suffixes=(None, "_extra"),
    sort=False
).convert_dtypes()
all_matches.head()

,VBO id,term label,DADIS_name,DADIS_species_name,dadis_transboundary_id,dadis_species_id,_merge,dadis_transboundary_id_extra,dadis_species_id_extra
0,VBO:0000012,Dromedary Bactrian Camel,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>
1,VBO:0000038,Alpaca (Alpaca),Alpaca,Alpaca,<NA>,<NA>,left_only,<NA>,<NA>
2,VBO:0000039,Huacaya (Alpaca),Huacaya,Alpaca,1-1,1,both,1-1,1
3,VBO:0000040,Suri (Alpaca),Suri,Alpaca,1-2,1,both,1-2,1
4,VBO:0000041,American Bison (American Bison),American Bison,American Bison,2-1,2,both,<NA>,<NA>


In [28]:
all_matches.pipe(lambda df: df.loc[df["dadis_transboundary_id"].isna() & ~ df["dadis_transboundary_id_extra"].isna(), :]).head()

,VBO id,term label,DADIS_name,DADIS_species_name,dadis_transboundary_id,dadis_species_id,_merge,dadis_transboundary_id_extra,dadis_species_id_extra
42,VBO:0000078,Nili-Ravi (Buffalo),Nili-Ravi,Buffalo,<NA>,<NA>,left_only,5-11,5
53,VBO:0000090,Aberdeen-Angus (Cattle),Aberdeen-Angus,Cattle,<NA>,<NA>,left_only,7-2,7
59,VBO:0000096,Ala-Tau (Cattle),Ala-Tau,Cattle,<NA>,<NA>,left_only,7-7,7
67,VBO:0000104,Angus (Cattle),Angus,Cattle,<NA>,<NA>,left_only,7-2,7
68,VBO:0000104,Angus (Cattle),Angus,Cattle,<NA>,<NA>,left_only,7-7,7


In [29]:
simple_ids = all_matches["dadis_transboundary_id"]
extra_ids = all_matches["dadis_transboundary_id_extra"]
all_matches["dadis_transboundary_id"] = simple_ids.where(~ simple_ids.isna(), extra_ids)

In [30]:
all_matches["dadis_transboundary_id"].isna().value_counts()

dadis_transboundary_id
False    1076
True      619
Name: count, dtype: int64

In [31]:
all_matches.head()

,VBO id,term label,DADIS_name,DADIS_species_name,dadis_transboundary_id,dadis_species_id,_merge,dadis_transboundary_id_extra,dadis_species_id_extra
0,VBO:0000012,Dromedary Bactrian Camel,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>
1,VBO:0000038,Alpaca (Alpaca),Alpaca,Alpaca,<NA>,<NA>,left_only,<NA>,<NA>
2,VBO:0000039,Huacaya (Alpaca),Huacaya,Alpaca,1-1,1,both,1-1,1
3,VBO:0000040,Suri (Alpaca),Suri,Alpaca,1-2,1,both,1-2,1
4,VBO:0000041,American Bison (American Bison),American Bison,American Bison,2-1,2,both,<NA>,<NA>


In [32]:
vbo_matched = vbo_transboundary.merge(
    all_matches[["VBO id", "dadis_transboundary_id"]],
    on="VBO id",
    how="left",
    sort=False
)
vbo_matched.head()

,VBO id,term label,breed name to be used for label,species to be used for label,internal term merge indicator vboid into vboid,internal date and issue separated by pipe YYMMDD-issue,GH issue NOT merge obsolete,parent ID,source for parents,source for parents.1,...,replacement label,comment.1,GH issue,obsolescence reason,curator note,DADIS_name,DADIS_species_name,DADIS_country,DADIS_ISO3_code,dadis_transboundary_id
0,VBO:0000012,Dromedary Bactrian Camel,<NA>,<NA>,<NA>,<NA>,<NA>,NCBITaxon:9836,<NA>,DOI:10.1016/j.jasrep.2021.103038|PMID:31969478...,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,VBO:0000038,Alpaca (Alpaca),Alpaca,Alpaca,<NA>,<NA>,<NA>,NCBITaxon:30538,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,Alpaca,Alpaca,<NA>,<NA>,<NA>
2,VBO:0000039,Huacaya (Alpaca),Huacaya,Alpaca,<NA>,<NA>,<NA>,NCBITaxon:30538,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,Huacaya,Alpaca,<NA>,<NA>,1-1
3,VBO:0000040,Suri (Alpaca),Suri,Alpaca,<NA>,<NA>,<NA>,NCBITaxon:30538,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,Suri,Alpaca,<NA>,<NA>,1-2
4,VBO:0000041,American Bison (American Bison),American Bison,American Bison,<NA>,<NA>,<NA>,NCBITaxon:9901,https://www.fao.org/dad-is,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,American Bison,American Bison,<NA>,<NA>,2-1
